In [3]:
import pandas as pd
import os
import numpy as np



In [4]:
figure_dir='./figures/'
data_dir = './data_kantar/'

In [5]:
transaction_limits = [1, 20, 100, 1500, np.inf]
intervals = list(zip(transaction_limits[:-1], transaction_limits[1:]))

In [6]:

year=2022
t=intervals[0]
def get_name(year,state,interval):
    return f'{year}_{state}_{interval[0]}-{interval[1]}.dta'

get_name(year,'raw',t)

'2022_raw_1-20.dta'

In [7]:
dfs={}
for t in intervals:
    dfs[t]=pd.read_stata(data_dir+get_name(year,'o',t))

In [8]:
df_shops = pd.read_stata(data_dir+f'{year}_shops.dta') 
df_products = pd.read_stata(data_dir+f'{year}_products.dta') 


In [20]:
df_customers = pd.read_stata(data_dir+f'{year}_customers.dta') 


In [9]:
df_shops.head()

,index,CodLugarCompra,Lugar_Compra,Canal
0,0,79,Mercadona,SUPER+AUTOS
1,1,28,CONDIS,SUPER+AUTOS
2,2,33,DIA,DISCOUNT
3,462,121,AUTOSERVICIO,SUPER+AUTOS
4,463,123,TIENDAS,TDA.DE ALIMENTACION


In [14]:
duplicates_in_shops = df_shops['CodLugarCompra'].duplicated().any()

False

In [15]:
df_products.head()

,index,CodProducto,CodigoBarras,CodVF,VF,SA7_BPL,SA4_Fabricante,SA3_Marca,SA2_Submarca,X102_Aditivos,...,X558_Variedad_Queso,X571_Peso_Bebe,X674_Tipo_Helado,X901_Localizacion,X902_Tipo_Producto,X903_Envase,X904_Variedad_Sabor,X5_Num_Tot_Unidades,X18_Num_Tot_Paquetes_Botes,X66_Preferred_Unit_Measure
0,0,791031,8480000105424,30,LECHE LARGA CONSERVACION,1,MERCADONA S.A.,HACENDADO,HACENDADO,SIN ADITIVO,...,nan,nan,nan,ESTANTERIA,LECHE,BOTELLA DE PLASTICO,NATURAL,1,0,1500
1,1,675179,8410702020886,524,V.C.D. ESPAÑOLA,1,CONDIS SUPERMERCATS S.A.,TRIVIUM,TRIVIUM,nan,...,nan,nan,nan,ESTANTERIA,TORO,BOTELLA CRISTAL,TINTA DE TORO,1,0,750
2,2,204305,SSA1410202,10,FRUTA FRESCA A PESO (CB),1,NINGUNA MARCA,NINGUNA MARCA,NINGUNA MARCA,nan,...,nan,nan,nan,nan,nan,nan,nan,0,0,1
3,6,290307,SSA1070201,10,FRUTA FRESCA A PESO (CB),1,NINGUNA MARCA,NINGUNA MARCA,NINGUNA MARCA,nan,...,nan,nan,nan,nan,nan,nan,nan,0,0,1
4,7,290355,SSA1130202,10,FRUTA FRESCA A PESO (CB),1,NINGUNA MARCA,NINGUNA MARCA,NINGUNA MARCA,nan,...,nan,nan,nan,nan,nan,nan,nan,0,0,1


In [42]:
dfp1=df_products.drop('index',axis=1)

In [43]:
dfp1=dfp1.drop_duplicates()

In [44]:
dfp1['CodProducto'].duplicated().any()



True

In [49]:
product_duplicates=dfp1[dfp1['CodProducto'].duplicated(keep=False)].sort_values('CodProducto')


In [51]:
d_records= product_duplicates[product_duplicates['CodProducto'] == product_duplicates.iloc[0]['CodProducto']]

In [55]:
d_records

,CodProducto,CodigoBarras,CodVF,VF,SA7_BPL,SA4_Fabricante,SA3_Marca,SA2_Submarca,X102_Aditivos,X108_Usuario,...,X558_Variedad_Queso,X571_Peso_Bebe,X674_Tipo_Helado,X901_Localizacion,X902_Tipo_Producto,X903_Envase,X904_Variedad_Sabor,X5_Num_Tot_Unidades,X18_Num_Tot_Paquetes_Botes,X66_Preferred_Unit_Measure
126326,13755,E8A14003010704650308,834,OTROS QUESOS (CB),1,QUESERIAS ENTREPINARES S.A.,ENTREPINARES,ENTREPINARES,nan,nan,...,OVEJA,nan,nan,nan,nan,nan,nan,0,0,1
127830,13755,E8A14002010704650308,834,OTROS QUESOS (CB),1,QUESERIAS ENTREPINARES S.A.,ENTREPINARES,ENTREPINARES,nan,nan,...,OVEJA,nan,nan,nan,nan,nan,nan,0,0,1


In [54]:
#_records.diff(periods=1,axis=0)
for c in d_records.columns:
    if d_records.iloc[0][c]!=d_records.iloc[1][c]:
        print(f'{c}: {d_records.iloc[0][c]}!={d_records.iloc[1][c]}')

CodigoBarras: E8A14003010704650308!=E8A14002010704650308


In [21]:
df_customers.head()

,index,CodPanelista,CodIndividuo,CC_AA,Edad_Ama,NF,Numero_Gatos,Numero_Perros,Presencia_Ni_os,CodIMC,IMC,Ponderacion,Ciclo_Vida,Clase_EGM,Clase_Social,Habitat_Metropolitano,Habitat_Municipal_Std,Inmigrante,Provincia,Region
0,0,364917,0,Com. Madrid,47,1,0,0,No,3,Sobrepeso,2,Adultos Independientes,Media,Media,AM Madrid,más 1000000,1,Madrid,MDD AM
1,1,30109,1,Andalucia,62,3,0,0,No,3,Sobrepeso,0,Parejas Adultas sin hijos,Media Alta,Media,No Metrop,20001 a 30000,1,Jaen,Andalucia
2,2,30109,0,Andalucia,62,3,0,0,No,3,Sobrepeso,0,Parejas Adultas sin hijos,Media Alta,Media,No Metrop,20001 a 30000,1,Jaen,Andalucia
3,133,30109,2,Andalucia,62,3,0,0,No,3,Sobrepeso,0,Parejas Adultas sin hijos,Media Alta,Media,No Metrop,20001 a 30000,1,Jaen,Andalucia
4,1883,30131,1,Andalucia,62,3,0,1,No,2,Normal,1,Parejas con hijos mayores,Media,Media baja,AM Sevilla,500001 a 1000000,1,Sevilla,Andalucia


In [23]:
duplicates_in_customers = df_customers['CodPanelista'].duplicated().any()


In [24]:
duplicates_in_customers

True

In [29]:
df_customers.columns

Index(['index', 'CodPanelista', 'CodIndividuo', 'CC_AA', 'Edad_Ama', 'NF',
       'Numero_Gatos', 'Numero_Perros', 'Presencia_Ni_os', 'CodIMC', 'IMC',
       'Ponderacion', 'Ciclo_Vida', 'Clase_EGM', 'Clase_Social',
       'Habitat_Metropolitano', 'Habitat_Municipal_Std', 'Inmigrante',
       'Provincia', 'Region'],
      dtype='object')

In [39]:
dfc1=df_customers.drop(['CodIndividuo','index'],axis=1)

In [40]:
dfc1=dfc1.drop_duplicates()

In [41]:
dfc1['CodPanelista'].duplicated().any()


False

In [34]:
duplicates_in_customers

True

In [37]:
dfc1[dfc1['CodPanelista'].duplicated(keep=False)]

,index,CodPanelista,CC_AA,Edad_Ama,NF,Numero_Gatos,Numero_Perros,Presencia_Ni_os,CodIMC,IMC,Ponderacion,Ciclo_Vida,Clase_EGM,Clase_Social,Habitat_Metropolitano,Habitat_Municipal_Std,Inmigrante,Provincia,Region
1,1,30109,Andalucia,62,3,0,0,No,3,Sobrepeso,0,Parejas Adultas sin hijos,Media Alta,Media,No Metrop,20001 a 30000,1,Jaen,Andalucia
2,2,30109,Andalucia,62,3,0,0,No,3,Sobrepeso,0,Parejas Adultas sin hijos,Media Alta,Media,No Metrop,20001 a 30000,1,Jaen,Andalucia
3,133,30109,Andalucia,62,3,0,0,No,3,Sobrepeso,0,Parejas Adultas sin hijos,Media Alta,Media,No Metrop,20001 a 30000,1,Jaen,Andalucia
4,1883,30131,Andalucia,62,3,0,1,No,2,Normal,1,Parejas con hijos mayores,Media,Media baja,AM Sevilla,500001 a 1000000,1,Sevilla,Andalucia
5,1913,30131,Andalucia,62,3,0,1,No,2,Normal,1,Parejas con hijos mayores,Media,Media baja,AM Sevilla,500001 a 1000000,1,Sevilla,Andalucia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13999,7580165,401917,Castilla la mancha,31,4,0,0,Niños 0-5,4,Obesidad,2,Parejas con hijos pequeños,Alta,Media,No Metrop,10001 a 20000,1,Ciudad Real,Rto Centro
14001,7581557,439823,Comunitat Valenciana,47,4,1,5,Niños 6-15,3,Sobrepeso,1,Parejas con hijos edad media,Media Alta,Media,No Metrop,20001 a 30000,1,Valencia,Levante
14002,7581560,439823,Comunitat Valenciana,47,4,1,5,Niños 6-15,3,Sobrepeso,1,Parejas con hijos edad media,Media Alta,Media,No Metrop,20001 a 30000,1,Valencia,Levante
14003,7581563,439823,Comunitat Valenciana,47,4,1,5,Niños 6-15,3,Sobrepeso,1,Parejas con hijos edad media,Media Alta,Media,No Metrop,20001 a 30000,1,Valencia,Levante
